## Sparkをはじめてみよう
Sparkを使って、README.mdを読み込み、いろいろと操作してみます。

In [ ]:
# scはSparkContextといって、sparkの基本となるオブジェクト。
textFile = sc.textFile("../README.md")

In [ ]:
# 行数をカウント
textFile.count()

In [ ]:
# "Spark"を含む行を抽出。
sparkLines = textFile.filter(lambda l: "Spark" in l)
sparkLines.count()

### filter-> pythonのfilterと同じ

In [ ]:
# "Spark"を含む行を全件取得
sparkLines.collect()

In [ ]:
# 出現単語ごとの件数を取得
textFile.flatMap(
    lambda l: l.strip().split()  # 行を単語に分解
).map(
    lambda w: (w, 1)  # 単語の数をカウントするため、（単語、1）というタプルに変換
).reduceByKey(
    lambda x, y: x + y  # 単語ごとに集計
).collect()

### flatMap -> mapしつつ、内部のコレクションを展開する関数。[[1,2], [3,4], [4,5]]をflatMapすると、[1,2,3,4,5]になる。
### map -> pythonのmapと同じ
### reduceByKey -> (key, value)のコレクションに対して、キーごとに値を集計する。

## RDDの作り方
RDDの作り方は、大きく分けて２通りあります。

In [ ]:
# コレクションオブジェクトから作成する
l = [1, 2, 3]
rdd = sc.parallelize(l)
rdd.collect()

In [ ]:
# 外部から読み込む
rdd = sc.textFile("../README.md")
rdd.count()

## RDDの処理イメージ
RDDのTransformationとActionがどのように実行されるのか見てみます。

In [ ]:
# テキストファイルからRDD作成。この時点ではRDDオブジェクトが作成されるだけで、実際の読み込みは発生しない。
linesRDD = sc.textFile("../README.md")
print type(linesRDD)

In [ ]:
# "Spark"というワードを持つ行のみ抽出。ここでも新たなRDDオブジェクトが作成されるだけ。
filteredRDD =  linesRDD.filter(lambda l: "Spark" in l)
print type(filteredRDD)

In [ ]:
# 行数取得。ここでやっとテキストファイルから読み込んでフィルタするという操作が実行される。
cnt = filteredRDD.count()
print cnt

## RDDのキャッシュ

In [ ]:
textFile = sc.textFile("../README.md")

In [ ]:
# cache()を呼び出すことで、メモリにキャッシュすることが指示されます。ここではまだキャッシュされません。
cached = textFile.filter(lambda l: "Spark" in l).cache()

In [ ]:
# ここで、テキストファイルからの読み込みとフィルタ処理が行われた上で、結果がメモリにキャッシュされます。
cached.count()

In [ ]:
# メモリからfilter処理までが終わった状態のデータを読み込み、処理します。テキストファイルが再び読み込まれることはありません。
import datetime
cached.saveAsTextFile("README-filtered-{}.md".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S")))

## Sparkの管理画面で実行を確認する

### Shuffleのないケース

In [ ]:
sc.textFile("../licenses/*.txt").filter(lambda l: "Copyright" in l).first()

[管理画面で確認](http://localhost:4040/)

### Shuffleのあるケース

In [ ]:
sc.textFile("../licenses/*.txt").flatMap(lambda l: l.split()).map(lambda w: (w, 1)).reduceByKey(lambda x, y: x + y).take(1)

[管理画面で確認](http://localhost:4040/)

## DataFrameを動かしてみよう

In [ ]:
# DataFrameを使うためのエントリーポイントとなるオブジェクト作成
from pyspark.sql import SparkSession
spark =  SparkSession.builder.appName("pycon2016").getOrCreate()

In [ ]:
# 下記のようなjsonがある
%cat ../examples/src/main/resources/people.json

In [ ]:
# これを読み込む
df = spark.read.json("../examples/src/main/resources/people.json")
df.show()

In [ ]:
# 特定のカラムを選択
df.select("name").show()

In [ ]:
# 絞り込み選択
df.filter(df['age'] > 21).show()

In [ ]:
# 集約関数
df.groupBy("age").count().show()

In [ ]:
# メモリ上にテンポラリテーブルとして保存することも可能。
df.createOrReplaceTempView("people")

In [ ]:
# テーブルには直接SQLを発行することができます。戻り値はDataFrame。
res = spark.sql("select age from people where name = 'Andy'")
res.show()

In [ ]:
# なお、スキーマについてはデータから類数されています。
df.printSchema()

In [ ]:
# もちろん、データを書き込むことも可能。jsonで読んだデータをcsvで書き込む。
newDf = df.select(df["name"], df["age"] + 10)
newDf.write.mode("overwrite").csv("/tmp/new_people.csv")
spark.read.csv("/tmp/new_people.csv").show()

## 様々なデータソース

In [ ]:
# csv
dfCsv = spark.read.csv("../examples/src/main/resources/test.csv")
dfCsv.show()

In [ ]:
# parquet
dfParquet = spark.read.parquet("../examples/src/main/resources/users.parquet")
dfParquet.show()

## DataFrameを使うメリット

In [ ]:
from pyspark.sql import Row
import pyspark.sql.functions as func

# 元データ
# (名前、部署名、年齢)
raw = [("Pete", "dev", 20), ("Keith", "dev", 22),  ("Roger", "sales", 30), ("John", "sales", 28)]
rdd = sc.parallelize(raw)
df = spark.createDataFrame(
    rdd.map(
        lambda r: Row(name=r[0], department=r[1], age=r[2])))

In [ ]:
# RDDで処理するケース
rdd.map(
    lambda r: (r[1], (float(r[2]), 1))  # (部署名、(年齢、カウント))のタプルに変換
).reduceByKey(
    lambda x, y: (x[0] + y[0], x[1] + y[1])  # 部署ごとに年齢と人数の合計を算出
).mapValues(
    lambda v: v[0] / v[1]  # 年齢の合計と人数の合計割り算
).collect()

In [ ]:
# DataFrameで処理するケース
df.groupBy("department").agg(func.avg("age")).show()

## メールのスパム判定

In [ ]:
# まずは元データの中身を見る。
sc.textFile("../data/smsspam/SMSSpamCollection").take(3)

# ラベルと文章からなるtsvファイル

In [ ]:
from pyspark.sql.types import *

# オリジナルデータをロード
# スキーマを個別で指定したい場合は、以下のようにStructTypeを作成する。
schema = StructType([StructField("label", StringType(), True), StructField("body", StringType(), True)])
df = spark.read.csv("../data/smsspam/SMSSpamCollection", schema=schema, sep="\t")

In [ ]:
from pyspark.sql.functions import when

# ham/spamといったラベルを0/1に変換
labeled = df.select(
    when(df["label"] == "ham", 0).otherwise(1)
    .cast(IntegerType())
    .alias("label"),
    df["body"])

# トレーニングデータとテストデータに分割
training, test = labeled.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

# 以下の流れのパイプラインを構築する。
# 文章を単語に分割 => 単語の選別 => 単語の出現頻度カウント => ロジスティック回帰モデル作成

# まずは、パイプラインのそれぞれのステージを構築するオブジェクトを作成する。
## 文章を単語に分割するやつ
tokenizer = Tokenizer(inputCol="body", outputCol="words")

## 単語を選別するやつ
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")

## 単語を数えるやつ
hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="features")

## ロジスティック回帰モデルを作成するやつ
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [ ]:
# 上記のオブジェクトを組み合わせてパイプラインを構築。
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, lr])

In [ ]:
# 学習モデル構築
model = pipeline.fit(training)

In [ ]:
# モデルの精度（AUC）を計測
from pyspark.ml.evaluation import BinaryClassificationEvaluator

prediction = model.transform(test)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")
evaluator.evaluate(prediction)